In [2]:
from os.path import isfile, join, exists
from os import listdir, makedirs
import numpy as np
import matplotlib.pyplot as plt
from brainpipe.system import study
from scipy import stats
from itertools import product
import pandas as pd

### Take only electrodes present at encoding AND retrieval
all common electrodes or only significant common electrodes

In [ ]:
from os import listdir
import numpy as np
import pandas as pd

th = '0.01'
freqs = ['0_theta', '1_alpha','2_beta','3_gamma']
rois_to_keep = ['ACC','Amg','Amg-PirT','HC','IFG','Ins','MFG','OFC','PHG','SFG','pPirT']
nb_elecs = 120+55+55
###############################################################################
st = study('Olfacto')
path_enc = join(st.path, 'feature/0_Power_Encoding_EpiPerf_By_Odor/')
path_ret = join(st.path, 'feature/0_Power_Retrieval_EpiPerf_By_Odor/')
path_E = join(st.path, 'figure/0_Classif_Power_E_EpiPerf_LowHigh_1000perm_BBG/')
path_R = join(st.path, 'figure/0_Classif_Power_R_EpiPerf_LowHigh_1000perm_BBG/')
npz_form = 'All_subjects_sources_{}_odor_low_high_sel_physFT.npz'
mask_form = 'All_subjects_mask_stat_{}_minwin1.0_th'+th+'.npy'
path2save = join(st.path, 'feature/TPSim_Enc_Ret_By_Odor/')
###############################################################################

for freq in freqs:
    #select all electrodes from encoding (specific regions and subjects)
    mat_enc = np.load(path_E+npz_form.format(freq))
    idx_E = np.where([roi in rois_to_keep for roi in mat_enc['s_labels']])[0]
    id_su = np.where([su != 'S6' for su in mat_enc['su_codes'][idx_E]])[0]
    xyz0, labels0 = mat_enc['s_xyz'][idx_E][id_su], mat_enc['s_labels'][idx_E][id_su]
    mask0 = np.load(path_npz_enc+'masks_stat/'+mask_form.format(freq))[id_su]
    mask0 = np.asarray([not x for x in mask0])
        
    #select all electrodes from retrieval (specific regions, all subjects)
    mat_ret = np.load(path_R+npz_form.format(freq))
    idx_R = np.where([roi in rois_to_keep for roi in mat_ret['s_labels']])[0]
    mask1 = np.load(path_npz_ret+'masks_stat/'+mask_form.format(freq))
    mask1 = np.asarray([not x for x in mask1])
    xyz1, labels1 = mat_ret['s_xyz'][idx_R], mat_ret['s_labels'][idx_R]
    
    #select only electrodes present both during encoding and retrieval
    to_keep_enc = np.where((xyz0==xyz1[:,None]).all(-1))[1]
    to_keep_ret = np.where((xyz1==xyz0[:,None]).all(-1))[1]
    mask0_new, mask1_new = mask0[to_keep_enc], mask1[to_keep_ret]
    print(len(mask0_new),len(mask1_new))
    
    xyz_shared = xyz0[to_keep_enc]
    xyz0_sig = xyz0[to_keep_enc][mask0_new]
    xyz1_sig = xyz1[to_keep_ret][mask1_new]
    print(freq,xyz0_sig.shape,xyz1_sig.shape)
            
    #create an array of region labels for encoding and retrieval
    dict_regions ={ 'ACC':'Frontal','IFG':'Frontal','MFG':'Frontal','SFG':'Frontal',
                'Amg':'Olf','pPirT':'Olf','Amg-PirT':'Olf','Ins':'Olf','OFC':'Olf',
                'HC':'MTL','PHG':'MTL'}
    regions0 = np.vectorize(dict_regions.get)(labels0[to_keep_enc])
        
    #loop on all elecs and create a code indicating when it decodes
    codes_elecs, region_elecs = [], []
    for i, elec in enumerate(xyz_shared):
        in_1 = np.where(np.all(elec == xyz0_sig, 1))[0] #encoding
        in_2 = np.where(np.all(elec == xyz1_sig, 1))[0] #retrieval
        
        if in_1.size and in_2.size: #decode in both
            codes_elecs.append(2)
            region_elecs.append(regions0[i])
        elif in_1.size and not in_2.size: #decode only in encoding
            codes_elecs.append(0)
            region_elecs.append(regions0[i])
        elif not in_1.size and in_2.size: #decode only in retrieval
            codes_elecs.append(1)
            region_elecs.append(regions0[i])
    
    codes_elecs = np.asarray(codes_elecs)[np.newaxis]
    region_elecs = np.asarray(region_elecs)[np.newaxis]
    count = np.ones(shape=region_elecs.shape)
    #print(freq,'regions',region_elecs.shape,codes_elecs.shape, count.shape)
    concat = np.concatenate((codes_elecs,region_elecs,count),axis=0).swapaxes(0,1)
    df = pd.DataFrame(concat,columns=['codes_elecs','region_elecs','count'])
    df_gr = df[['codes_elecs','region_elecs']].groupby(['codes_elecs']).count()
    codes = df_gr.index.get_level_values(0).values
    rois = df_gr.columns.get_level_values(0).values
    print(df_gr, codes, rois)


In [ ]:

if not exists(path2save):
    makedirs(path2save)
###############################################################################
sel = 'signif' #'all'
freqs = ['2_theta', '3_alpha', '4_beta','5_gamma1','6_gamma2']
method = ['s_Mail_RL']
odors_su = {'CHAF': {5:12,7:68,8:36,9:96,1:6,2:2,3:68,4:8},
            'LEFC': {1:4,2:0,3:6,4:12,14:96,15:2,16:4,17:68},
            'PIRJ': {4:36,9:2,1:4,18:32,6:34,5:4,7:68}, #missing odor 15
            'VACJ': {14:6,15:64,16:68,17:8,10:6,11:4,12:4,13:40},
            'SEMC': {10:2,11:6,12:6,13:6,5:8,7:4,8:8,9:10},
            'FERJ': {16:6,17:6,5:8,7:6,12:8,13:8,2:6,1:10}}
rois_to_keep = ['ACC','Amg','Amg-PirT','HC','IFG','Ins','MFG','OFC','PHG',
                'SFG','pPirT']
th = 0.01
s_E, s_R = [0,1,2,4,5,6], range(6)

for s0, s1, su in zip(s_E,s_R,sorted(odors_su)):
    for odor in odors_su[su]:        
        if sel == 'signif': #select only significant electrodes at Encoding & Retrieval
            for f,freq in enumerate(freqs):
                mat0 = np.load(path_enc+su+'_odor_'+str(odor)+'_bipo_sel_physFT_pow.npz')
                mat1 = np.load(path_ret+su+'_odor_'+str(odor)+'_bipo_sel_physFT_pow.npz')
                data0, channels0 = mat0['xpow'], mat0['channels']
                data1, channels1 = mat1['xpow'], mat1['channels']
                #print(su,'num su',s0,s1, odor,freq,'data shape: ', data0.shape, data1.shape,'channels',channels0.shape,
                  #channels1.shape)

                #Select specific ROIs
                idx_roisE = np.where([roi in rois_to_keep for roi in mat0['Mai_RL']])[0]
                idx_roisR = np.where([roi in rois_to_keep for roi in mat1['Mai_RL']])[0]
                data0, channels0 = data0[:,idx_roisE,:,:], channels0[idx_roisE]
                data1, channels1 = data1[:,idx_roisR,:,:], channels1[idx_roisR]
                print('selec shape: ', data0.shape, data1.shape,'channels',channels0.shape,
                  channels1.shape)
                
                # Select one's subject ROIs
                df_E = pd.read_csv(path_maskE+'0_all_subjects_info_elecs.csv')
                df_R = pd.read_csv(path_maskR+'0_all_subjects_info_elecs.csv')
                                
                id_elecsE = np.where(df_E['subjects'] == 'S'+str(s0))[0]
                id_elecsR = np.where(df_R['subjects'] == 'S'+str(s1))[0]
                #print('elecs belonging to',su,id_elecsE.shape,id_elecsR.shape)
                maskE = np.load(path_maskE+'{}/All_subjects_mask_stat_{}_minwin1.0_th{}.npy'.format('masks_stat',freq,th))
                maskR = np.load(path_maskR+'{}/All_subjects_mask_stat_{}_minwin1.0_th{}.npy'.format('masks_stat',freq,th))
                #print('masks shapes', maskE.shape,maskR.shape)
                maskE,maskR = maskE[id_elecsE], maskR[id_elecsR]
                #print('mask for',su,maskE.shape,maskR.shape) #mask based on ROIs already
                channels0, channels1 = channels0[maskE], channels1[maskR]
                
                #create a list of electrodes to keep
                to_keep_enc = [idx for idx,elec in enumerate(channels0) if elec in channels1]
                to_keep_ret = [idx for idx,elec in enumerate(channels1) if elec in channels0]

                #select power files for to keep electrodes
                data0_new = data0[f,to_keep_enc,:,:]
                data1_new = data1[f,to_keep_ret,:,:]
                channels0_new = channels0[to_keep_enc]
                channels1_new = channels1[to_keep_ret]
                print('SIG elecs E/R for',su,freqs[f],channels0_new.shape, channels1_new.shape)
                labels0_new = mat0['Mai_RL'][idx_roisE][to_keep_enc]
                labels1_new = mat1['Mai_RL'][idx_roisR][to_keep_ret]
                xyz0_new = mat0['xyz'][idx_roisE][to_keep_enc]
                xyz1_new = mat1['xyz'][idx_roisR][to_keep_ret]

                #sorted all values from encoding and retrieval
                sorted_idx0 = np.argsort(channels0_new)
                data0_new = data0_new[sorted_idx0,:,:]
                channels0_new = channels0_new[sorted_idx0]
                labels0_new = labels0_new[sorted_idx0]
                xyz0_new = xyz0_new[sorted_idx0]
                sorted_idx1 = np.argsort(channels1_new)
                data1_new = data1_new[sorted_idx1,:,:]
                channels1_new = channels1_new[sorted_idx1]
                labels1_new = labels1_new[sorted_idx1]
                xyz1_new = xyz1_new[sorted_idx1]

                #Save all values from encoding and retrieval
                mat0_new = {'xpow':data0_new, 'label':labels0_new, 'channel':channels0_new,
                            'xyz':xyz0_new}
                mat1_new = {'xpow':data1_new, 'label':labels1_new, 'channel':channels1_new,
                            'xyz':xyz1_new}
                np.savez(path2save+su+'_odor'+str(odor)+'_E_pow{}.npz'.format(freq),**mat0_new)
                np.savez(path2save+su+'_odor'+str(odor)+'_R_pow{}.npz'.format(freq),**mat1_new)
        else:
            to_keep_enc = [idx for idx,elec in enumerate(channels0) if elec in channels1]
            to_keep_ret = [idx for idx,elec in enumerate(channels1) if elec in channels0]

            #select power files for to keep electrodes
            data0_new = data0[:,to_keep_enc,:,:]
            data1_new = data1[:,to_keep_ret,:,:]
            channels0_new = channels0[to_keep_enc]
            channels1_new = channels1[to_keep_ret]
            labels0_new = mat0['Mai_RL'][to_keep_enc]
            labels1_new = mat1['Mai_RL'][to_keep_ret]
            xyz0_new = mat0['xyz'][to_keep_enc]
            xyz1_new = mat1['xyz'][to_keep_ret]

            #sorted all values from encoding and retrieval
            sorted_idx0 = np.argsort(channels0_new)
            data0_new = data0_new[:,sorted_idx0,:,:]
            channels0_new = channels0_new[sorted_idx0]
            labels0_new = labels0_new[sorted_idx0]
            xyz0_new = xyz0_new[sorted_idx0]
            sorted_idx1 = np.argsort(channels1_new)
            data1_new = data1_new[:,sorted_idx1,:,:]
            channels1_new = channels1_new[sorted_idx1]
            labels1_new = labels1_new[sorted_idx1]
            xyz1_new = xyz1_new[sorted_idx1]

            #Save all values from encoding and retrieval
            mat0_new = {'xpow':data0_new, 'label':labels0_new, 'channel':channels0_new,
                        'xyz':xyz0_new}
            mat1_new = {'xpow':data1_new, 'label':labels1_new, 'channel':channels1_new,
                        'xyz':xyz1_new}
            np.savez(path2save+su+'_odor_'+str(odor)+'_E_pow.npz',**mat0_new)
            np.savez(path2save+su+'_odor_'+str(odor)+'_R_pow.npz',**mat1_new)

### Compute TPSim for common electrodes
By odor (Similarity btw E and R by odor)

In [13]:
from numpy.random import permutation
from brainpipe.statistics import *

###############################################################################
st = study('Olfacto')
path_npz = join(st.path,'feature/TPSim_power_data/Power_all_elecs_E_R_by_odor_all_elecsFT/')
path2save = join(st.path, 'feature/TPSim_Enc_Ret_By_Odor_By_Cond/TPS_by_odor/by_odor_wth/')
npz_enc = join(path_npz, '{}_odor_{}_E_6freqs.npz')
npz_ret = join(path_npz, '{}_odor_{}_R_6freqs.npz')
###############################################################################
if not exists(path2save):
    makedirs(path2save)
###############################################################################
freqs = ['delta','theta', 'alpha', 'beta', 'low_gamma','high_gamma'] # Name of each frequency
#freqs = ['2_theta', '3_alpha', '4_beta','5_gamma1','6_gamma2']
method = ['labels']
odors_su = {'CHAF': {5:12,7:68,8:36,9:96,1:6,2:2,3:68,4:8},
            'LEFC': {1:4,2:0,3:6,4:12,14:96,15:2,16:4,17:68},
            'PIRJ': {4:36,9:2,1:4,18:32,6:34,5:4,7:68}, #missing odor 15
            'VACJ': {14:6,15:64,16:68,17:8,10:6,11:4,12:4,13:40},
            'SEMC': {10:2,11:6,12:6,13:6,5:8,7:4,8:8,9:10},
            'FERJ': {16:6,17:6,5:8,7:6,12:8,13:8,2:6,1:10}}
to_take = [17,47]

for s,su in enumerate(sorted(odors_su)):
    for odor in sorted(odors_su[su]):
        for f,freq in enumerate(freqs):
            mat_enc = np.load(npz_enc.format(su,odor,freq))
            mat_ret = np.load(npz_ret.format(su,odor,freq))
            pow_enc = mat_enc['xpow'][f,:,to_take[0]:to_take[1],:]
            pow_ret = mat_ret['xpow'][f,:,to_take[0]:to_take[1],:]
            print('enc',pow_enc.shape,'ret',pow_ret.shape)
            R_freq, p_freq = np.array([]), np.array([])
            for elec in range(pow_enc.shape[0]):
                pow0 = pow_enc[elec,:]
                pow1 = pow_ret[elec,:]
                R_trials, p_trials = np.array([]), np.array([])
                for t0, t1 in product(range(pow0.shape[1]),range(pow1.shape[1])):
                    R, p = stats.pearsonr(pow0[:,t0],pow1[:,t1])
                    R_trials = np.vstack((R_trials,R)) if np.size(R_trials) else R
                    p_trials = np.vstack((p_trials,p)) if np.size(p_trials) else p
                R_freq = np.vstack((R_freq,R_trials.T)) if np.size(R_freq) else R_trials.T
                p_freq = np.vstack((p_freq,p_trials.T)) if np.size(p_freq) else p_trials.T
                #print(R_freq.shape, p_freq.shape)
            R_freq = np.arctanh(R_freq)
            print(su,odor,freq,'R',R_freq.shape, p_freq.shape)
            mat = {'tps':R_freq, 'pval':p_freq,'label':mat_enc['labels'],
                   'channel':mat_enc['channels'], 'xyz':mat_enc['xyz']}
            np.savez(path2save+'TPS_pears_'+su+'_odor'+str(odor)+'_wth_'+freq+'.npz',**mat)

-> Olfacto loaded
enc (57, 30, 1) ret (57, 30, 3)
CHAF 1 delta R (57, 3) (57, 3)
enc (57, 30, 1) ret (57, 30, 3)
CHAF 1 theta R (57, 3) (57, 3)
enc (57, 30, 1) ret (57, 30, 3)
CHAF 1 alpha R (57, 3) (57, 3)
enc (57, 30, 1) ret (57, 30, 3)
CHAF 1 beta R (57, 3) (57, 3)
enc (57, 30, 1) ret (57, 30, 3)
CHAF 1 low_gamma R (57, 3) (57, 3)
enc (57, 30, 1) ret (57, 30, 3)
CHAF 1 high_gamma R (57, 3) (57, 3)
enc (57, 30, 3) ret (57, 30, 3)
CHAF 2 delta R (57, 9) (57, 9)
enc (57, 30, 3) ret (57, 30, 3)
CHAF 2 theta R (57, 9) (57, 9)
enc (57, 30, 3) ret (57, 30, 3)
CHAF 2 alpha R (57, 9) (57, 9)
enc (57, 30, 3) ret (57, 30, 3)
CHAF 2 beta R (57, 9) (57, 9)
enc (57, 30, 3) ret (57, 30, 3)
CHAF 2 low_gamma R (57, 9) (57, 9)
enc (57, 30, 3) ret (57, 30, 3)
CHAF 2 high_gamma R (57, 9) (57, 9)
enc (57, 30, 3) ret (57, 30, 3)
CHAF 3 delta R (57, 9) (57, 9)
enc (57, 30, 3) ret (57, 30, 3)
CHAF 3 theta R (57, 9) (57, 9)
enc (57, 30, 3) ret (57, 30, 3)
CHAF 3 alpha R (57, 9) (57, 9)
enc (57, 30, 3) ret (

LEFC 14 high_gamma R (36, 18) (36, 18)
enc (36, 30, 11) ret (36, 30, 3)
LEFC 15 delta R (36, 33) (36, 33)
enc (36, 30, 11) ret (36, 30, 3)
LEFC 15 theta R (36, 33) (36, 33)
enc (36, 30, 11) ret (36, 30, 3)
LEFC 15 alpha R (36, 33) (36, 33)
enc (36, 30, 11) ret (36, 30, 3)
LEFC 15 beta R (36, 33) (36, 33)
enc (36, 30, 11) ret (36, 30, 3)
LEFC 15 low_gamma R (36, 33) (36, 33)
enc (36, 30, 11) ret (36, 30, 3)
LEFC 15 high_gamma R (36, 33) (36, 33)
enc (36, 30, 16) ret (36, 30, 3)
LEFC 16 delta R (36, 48) (36, 48)
enc (36, 30, 16) ret (36, 30, 3)
LEFC 16 theta R (36, 48) (36, 48)
enc (36, 30, 16) ret (36, 30, 3)
LEFC 16 alpha R (36, 48) (36, 48)
enc (36, 30, 16) ret (36, 30, 3)
LEFC 16 beta R (36, 48) (36, 48)
enc (36, 30, 16) ret (36, 30, 3)
LEFC 16 low_gamma R (36, 48) (36, 48)
enc (36, 30, 16) ret (36, 30, 3)
LEFC 16 high_gamma R (36, 48) (36, 48)
enc (36, 30, 7) ret (36, 30, 3)
LEFC 17 delta R (36, 21) (36, 21)
enc (36, 30, 7) ret (36, 30, 3)
LEFC 17 theta R (36, 21) (36, 21)
enc (36, 

VACJ 12 low_gamma R (48, 6) (48, 6)
enc (48, 30, 2) ret (48, 30, 3)
VACJ 12 high_gamma R (48, 6) (48, 6)
enc (48, 30, 4) ret (48, 30, 3)
VACJ 13 delta R (48, 12) (48, 12)
enc (48, 30, 4) ret (48, 30, 3)
VACJ 13 theta R (48, 12) (48, 12)
enc (48, 30, 4) ret (48, 30, 3)
VACJ 13 alpha R (48, 12) (48, 12)
enc (48, 30, 4) ret (48, 30, 3)
VACJ 13 beta R (48, 12) (48, 12)
enc (48, 30, 4) ret (48, 30, 3)
VACJ 13 low_gamma R (48, 12) (48, 12)
enc (48, 30, 4) ret (48, 30, 3)
VACJ 13 high_gamma R (48, 12) (48, 12)
enc (48, 30, 3) ret (48, 30, 3)
VACJ 14 delta R (48, 9) (48, 9)
enc (48, 30, 3) ret (48, 30, 3)
VACJ 14 theta R (48, 9) (48, 9)
enc (48, 30, 3) ret (48, 30, 3)
VACJ 14 alpha R (48, 9) (48, 9)
enc (48, 30, 3) ret (48, 30, 3)
VACJ 14 beta R (48, 9) (48, 9)
enc (48, 30, 3) ret (48, 30, 3)
VACJ 14 low_gamma R (48, 9) (48, 9)
enc (48, 30, 3) ret (48, 30, 3)
VACJ 14 high_gamma R (48, 9) (48, 9)
enc (48, 30, 2) ret (48, 30, 3)
VACJ 15 delta R (48, 6) (48, 6)
enc (48, 30, 2) ret (48, 30, 3)
VAC

#### BETWEEN SIMILARITY By odor (Similarity btw E and R by odor)

In [15]:
"""
Compute TPSim BTW odors By ODOR
"""
###############################################################################
st = study('Olfacto')
path_pow = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_odor/')
pow_file = join(path_pow, '{}_odor_{}_{}_6freqs.npz')
path2save = join(st.path, 'feature/TPSim_{}_By_Odor_By_Cond/TPS_by_odor/by_odor_btw/')
tps_save = join(path2save, 'TPS_pears_{}_odor{}_btw_{}.npz')
###############################################################################
exp = 'Enc_Ret' #Ret, Enc
shortcut = ['E','R'] if exp == 'Enc_Ret' else [exp[0]]
if not exists(path2save.format(exp)):
    makedirs(path2save.format(exp))
###############################################################################
freqs = ['delta','theta', 'alpha', 'beta','low_gamma','high_gamma']
odors_su = {'CHAF': {5:12,7:68,8:36,9:96,1:6,2:2,3:68,4:8},
            'LEFC': {1:4,2:0,3:6,4:12,14:96,15:2,16:4,17:68},
            'PIRJ': {4:36,9:2,1:4,18:32,6:34,5:4,7:68}, #missing odor 15
            'VACJ': {14:6,15:64,16:68,17:8,10:6,11:4,12:4,13:40},
            'SEMC': {10:2,11:6,12:6,13:6,5:8,7:4,8:8,9:10},
            'FERJ': {16:6,17:6,5:8,7:6,12:8,13:8,2:6,1:10}}

def tpsim_btw_odors_by_odor(su,o1,o2,f,freq):
    """
    Compute tpsim for one odor with all other odors in list od 2
    Parameters
    ----------
    su : string (subject name)
    od1, od2 : list
        o1 of length 1 and o2 of length (n_odors - 1)
    freq : string

    Returns
    -------
    tpsim : array
        The Temporal Pattern Similarity between all odors BY odor
        (n_combinations of trials btw o1 and o2)
    """
    mat_o1 = np.load(pow_file.format(su,o1,shortcut[0]),allow_pickle=True)
    print(mat_o1.files)
    pow_o1 = mat_o1['xpow'][f,:,17:47,:] #nelecs,npts,ntrials
    pow_o2 = np.array([])
    for o in o2:
        pow_ = np.load(pow_file.format(su,o,shortcut[1],freq))['xpow'][f,:,17:47,:]
        pow_o2 = np.concatenate((pow_o2,pow_),axis=-1) if np.size(pow_o2) else pow_
    R_freq, p_freq = np.array([]), np.array([])
    for elec in range(pow_o2.shape[0]):
        pow1, pow2 = pow_o1[elec], pow_o2[elec]
        R_trials, p_trials = np.array([]), np.array([])
        for t0, t1 in product(range(pow1.shape[-1]),range(pow2.shape[-1])):
            R, p = stats.pearsonr(pow1[:,t0],pow2[:,t1])
            R_trials = np.vstack((R_trials,R)) if np.size(R_trials) else R
            p_trials = np.vstack((p_trials,p)) if np.size(p_trials) else p
        R_freq = np.vstack((R_freq,R_trials.T)) if np.size(R_freq) else R_trials.T
        p_freq = np.vstack((p_freq,p_trials.T)) if np.size(p_freq) else p_trials.T
        #print(R_freq.shape, p_freq.shape)
    R_freq = np.arctanh(R_freq)
    print(su,o1,freq,'R',R_freq.shape, p_freq.shape,
              'o1 data',pow_o1.shape, 'o2 data',pow_o2.shape)
    mat = {'tps':R_freq, 'pval':p_freq,'label':mat_o1['labels'],
           'channel':mat_o1['channels'], 'xyz':mat_o1['xyz']}
    np.savez(tps_save.format(exp,su,o1,freq),**mat)
    
for su in odors_su:
    for o1, freq in product(odors_su[su],freqs):
        f = [i for i,f in enumerate(freqs) if f == freq][0]
        o2 = [o for o in odors_su[su] if o != o1]
        tpsim_btw_odors_by_odor(su,o1,o2,f,freq)

-> Olfacto loaded
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
CHAF 5 delta R (57, 21) (57, 21) o1 data (57, 30, 1) o2 data (57, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
CHAF 5 theta R (57, 21) (57, 21) o1 data (57, 30, 1) o2 data (57, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
CHAF 5 alpha R (57, 21) (57, 21) o1 data (57, 30, 1) o2 data (57, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
CHAF 5 beta R (57, 21) (57, 21) o1 data (57, 30, 1) o2 data (57, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
CHAF 5 low_gamma R (57, 21) (57, 21) o1 data (57, 30, 1) o2 data (57, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
CHAF 5 high_gam

CHAF 4 high_gamma R (57, 21) (57, 21) o1 data (57, 30, 1) o2 data (57, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
LEFC 1 delta R (36, 42) (36, 42) o1 data (36, 30, 2) o2 data (36, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
LEFC 1 theta R (36, 42) (36, 42) o1 data (36, 30, 2) o2 data (36, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
LEFC 1 alpha R (36, 42) (36, 42) o1 data (36, 30, 2) o2 data (36, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
LEFC 1 beta R (36, 42) (36, 42) o1 data (36, 30, 2) o2 data (36, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
LEFC 1 low_gamma R (36, 42) (36, 42) o1 data (36, 30, 2) o2 data (36, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'lab

LEFC 17 low_gamma R (36, 147) (36, 147) o1 data (36, 30, 7) o2 data (36, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
LEFC 17 high_gamma R (36, 147) (36, 147) o1 data (36, 30, 7) o2 data (36, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
PIRJ 4 delta R (30, 96) (30, 96) o1 data (30, 30, 4) o2 data (30, 30, 24)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
PIRJ 4 theta R (30, 96) (30, 96) o1 data (30, 30, 4) o2 data (30, 30, 24)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
PIRJ 4 alpha R (30, 96) (30, 96) o1 data (30, 30, 4) o2 data (30, 30, 24)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
PIRJ 4 beta R (30, 96) (30, 96) o1 data (30, 30, 4) o2 data (30, 30, 24)
['f', 'split', 'width', 'step', 'time', 'fname'

VACJ 14 beta R (48, 63) (48, 63) o1 data (48, 30, 3) o2 data (48, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
VACJ 14 low_gamma R (48, 63) (48, 63) o1 data (48, 30, 3) o2 data (48, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
VACJ 14 high_gamma R (48, 63) (48, 63) o1 data (48, 30, 3) o2 data (48, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
VACJ 15 delta R (48, 42) (48, 42) o1 data (48, 30, 2) o2 data (48, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
VACJ 15 theta R (48, 42) (48, 42) o1 data (48, 30, 2) o2 data (48, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
VACJ 15 alpha R (48, 42) (48, 42) o1 data (48, 30, 2) o2 data (48, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname'

SEMC 10 alpha R (62, 147) (62, 147) o1 data (62, 30, 7) o2 data (62, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
SEMC 10 beta R (62, 147) (62, 147) o1 data (62, 30, 7) o2 data (62, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
SEMC 10 low_gamma R (62, 147) (62, 147) o1 data (62, 30, 7) o2 data (62, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
SEMC 10 high_gamma R (62, 147) (62, 147) o1 data (62, 30, 7) o2 data (62, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
SEMC 11 delta R (62, 84) (62, 84) o1 data (62, 30, 4) o2 data (62, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
SEMC 11 theta R (62, 84) (62, 84) o1 data (62, 30, 4) o2 data (62, 30, 21)
['f', 'split', 'width', 'step', 'time',

FERJ 16 theta R (45, 63) (45, 63) o1 data (45, 30, 3) o2 data (45, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
FERJ 16 alpha R (45, 63) (45, 63) o1 data (45, 30, 3) o2 data (45, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
FERJ 16 beta R (45, 63) (45, 63) o1 data (45, 30, 3) o2 data (45, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
FERJ 16 low_gamma R (45, 63) (45, 63) o1 data (45, 30, 3) o2 data (45, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
FERJ 16 high_gamma R (45, 63) (45, 63) o1 data (45, 30, 3) o2 data (45, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
FERJ 17 delta R (45, 189) (45, 189) o1 data (45, 30, 9) o2 data (45, 30, 21)
['f', 'split', 'width', 'step', 'time', 'fnam

#### Compute TPSim values by Odor for High and Low conditions

In [ ]:
from numpy.random import permutation
from itertools import product
from brainpipe.statistics import *

###############################################################################
st = study('Olfacto')
exp, method = 'Ret', 'wth'
path_files = join(st.path,'feature/TPSim_'+exp+'_By_Odor_By_Cond/TPS_by_odor/')
filename = join(path_files, 'by_odor_'+method+'/TPS_pears_{}_odor_{}_{}.npz')
savename = join(path_files, 'TPS_pears_{}_{}_'+method+'_{}.npz')
###############################################################################
if not exists(path2save):
    makedirs(path2save)
###############################################################################
freqs = ['delta','theta','alpha','beta','low_gamma','high_gamma']
if exp != 'Enc' and method == 'wth':
    su_odor_groups = {'CHAF' : {'low':['1','2','4','5'],
                                  'high':['3','8','7','9']},
                        'VACJ' : {'low':['11','14','12','10'],
                                  'high':['15','17','16','13']},
                        'SEMC' : {'low':['7','10','11','12','13'],
                                  'high':['5','8','9']},
                        'PIRJ' : {'low':['1','9','5'],
                                  'high':['4','6','7','18']},
                        'LEFC' : {'low':['15','2','1','16'],
                                  'high':['14','3','4','17']},
                        'FERJ' : {'low':['7','2','16','17'],
                                  'high':['12','1','5','13']},}
else:
    su_odor_groups = {'CHAF' : {'low':['2'],
                              'high':['3','8','7','9']},
                    'VACJ' : {'low':['11','14','12','10'],
                              'high':['15','17','16','13']},
                    'SEMC' : {'low':['7','10','11','12','13'],
                              'high':['5','8','9']},
                    'PIRJ' : {'low':['1','9'],
                              'high':['4','18']},
                    'LEFC' : {'low':['15','2','1','16'],
                              'high':['14','3','4','17']},
                    'FERJ' : {'low':['7','2','16','17'],
                              'high':['12','1','5','13']},}

for su in (sorted(su_odor_groups)):
    for cond in su_odor_groups[su]:
        for freq in freqs:
            concat = np.array([])
            for odor in su_odor_groups[su][cond]:
                mat = np.load(filename.format(su,odor,freq),allow_pickle=True)
                concat = np.concatenate((concat,mat['tps']), axis=-1) if np.size(concat) else mat['tps']
            dico = {'tps':concat, 'label':mat['label'],'channel':mat['channel'],
                    'xyz':mat['xyz']}
            np.savez(savename.format(su,cond,freq),**dico)

### Compute TPSim within low and high conditions
1 - create power npz files for common electrodes by subj and freq <br> 
2 - compute the TPSim for Low and High conditions (all combinations) <br> 

In [ ]:
from numpy.random import permutation
from itertools import product
from brainpipe.statistics import *

###############################################################################
st = study('Olfacto')
path_files = join(st.path,'feature/TPSim_Enc_Ret_By_Odor_all/')
path2save = join(path_files, 'TPS_R_p_by_cond/')
###############################################################################
if not exists(path2save):
    makedirs(path2save)
###############################################################################
sessions = ['E','R']
su_codes = ['S0','S1','S2','S3','S4','S5']
freqs = ['0_theta', '1_alpha', '2_beta','3_gamma']
method = ['s_Mail_RL']
su_odor_groups = {'CHAF' : {'low':['1','2','4','5'],
                              'high':['3','8','7','9']},
                    'VACJ' : {'low':['11','14','12','10'],
                              'high':['15','17','16','13']},
                    'SEMC' : {'low':['7','10','11','12','13'],
                              'high':['5','8','9']},
                    'PIRJ' : {'low':['1','9','5'],
                              'high':['4','6','7','18']},
                    'LEFC' : {'low':['15','2','1','16'],
                              'high':['14','3','4','17']},
                    'FERJ' : {'low':['7','2','16','17'],
                              'high':['12','1','5','13']},}

for su in (sorted(su_odor_groups)):
    for sess in sessions:
        for cond in su_odor_groups[su]:
            big_concat = np.array([])
            for freq in freqs:
                concat = np.array([])
                for odor in su_odor_groups[su][cond]:
                    mat = np.load(path_files+su+'_odor'+str(odor)+'_'+sess+'_pow'+freq+'.npz',
                                 allow_pickle=True)
                    power = mat['xpow']
#                     print(su, freq, sess, cond, odor, mat.files)
                    concat = np.concatenate((concat,power), axis=2) if np.size(concat) else power
                #concat by freq
                big_concat = np.concatenate((big_concat,concat[np.newaxis]), axis=0) if np.size(big_concat) else concat[np.newaxis]
            #SAVE BIG_CONCAT FILE
            dico = {'xpow':big_concat, 'label':mat['label'],'channel':mat['channel'],'xyz':mat['xyz']}
            np.savez(path2save+'Pow_'+su+'_'+sess+'_'+cond+'_allfreqs.npz',**dico)

In [17]:
from numpy.random import permutation
from brainpipe.statistics import *

###############################################################################
RT_type = 'RT_rec_wght'#,'RT_epi_wght','RT_rec_first','RT_rec_wght','RT_epi_first
st = study('Olfacto')
path_files = join(st.path,'feature/')
path_npz = join(path_files,'TPSim_power_data/Power_all_elecs_E_R_by_cond_3groups_v=1_elecs=all/')
path2save = join(path_files,'TPSim_3groups_Enc_Ret/TPS_btw_v=1_elecs=all/')
#npz_enc = join(path_files, 'Encoding_EpiPerf_LowHigh/{}_odor_{}_common_renamed_OFC_HC.npz')
#npz_ret = join(path_files, 'Retrieval_EpiPerf_LowHigh/{}_odor_{}_common_renamed_OFC_HC.npz')
npz_enc = join(path_npz, '{}_odor_{}_E2_6freqs.npz')
npz_ret = join(path_npz, '{}_odor_{}_R_6freqs.npz')
savename = 'TPS_pears_{}_{}_btw_{}_E=all_R=all.npz'
###############################################################################
if not exists(path2save):
    makedirs(path2save)
###############################################################################
freqs = ['delta','theta', 'alpha', 'beta','low_gamma','high_gamma']
# freqs = ['0_theta', '1_alpha', '2_beta','3_gamma']
#method = ['s_Mail_RL']
conds = ['low','mid','high']
subjects = ['PIRJ','CHAF','FERJ','LEFC','VACJ','SEMC']
to_take = [17,47]

for su in subjects:
    for cond in conds:
        for f,freq in enumerate(freqs):
            mat_enc = np.load(npz_enc.format(su,cond),allow_pickle=True)
            mat_ret = np.load(npz_ret.format(su,cond),allow_pickle=True)
            pow_enc = mat_enc['xpow'][f,:,to_take[0]:to_take[1],:]
            pow_ret = mat_ret['xpow'][f,:,to_take[0]:to_take[1],:]
            print('enc',pow_enc.shape,'ret',pow_ret.shape)
            R_freq, p_freq = np.array([]), np.array([])
            for elec in range(pow_enc.shape[0]):
                pow0 = pow_enc[elec,...]
                pow1 = pow_ret[elec,...]
                R_trials, p_trials = np.array([]), np.array([])
                for t0, t1 in product(range(pow0.shape[1]),range(pow1.shape[1])):
    #                     print(t0,t1,pow0[:,t0].shape,pow1[:,t1].shape)
                    R, p = stats.pearsonr(pow0[:,t0],pow1[:,t1])
                    D = R # <<<<<<< HERE TO CHANGE FOR DISTANCE COMPUTATIONS
                    R_trials = np.vstack((R_trials,D)) if np.size(R_trials) else D
                    p_trials = np.vstack((p_trials,p)) if np.size(p_trials) else p
    #                     print(R_trials.shape,p_trials.shape)
                R_freq = np.vstack((R_freq,R_trials.T)) if np.size(R_freq) else R_trials.T
                p_freq = np.vstack((p_freq,p_trials.T)) if np.size(p_freq) else p_trials.T
    #             print(R_freq.shape, p_freq.shape)
            print(su,cond,'R',R_freq.shape, p_freq.shape)
            R_freq = 1 - np.arctanh(R_freq)
            mat = {'tps':R_freq, 'pval':p_freq,'label':mat_enc['labels'],
                   'channel':mat_enc['channels'], 'xyz':mat_enc['xyz']}
            np.savez(path2save+savename.format(su,cond,freq),**mat)

-> Olfacto loaded
enc (18, 30, 14) ret (18, 30, 12)
PIRJ low R (18, 168) (18, 168)
enc (18, 30, 14) ret (18, 30, 12)
PIRJ low R (18, 168) (18, 168)
enc (18, 30, 14) ret (18, 30, 12)
PIRJ low R (18, 168) (18, 168)
enc (18, 30, 14) ret (18, 30, 12)
PIRJ low R (18, 168) (18, 168)
enc (18, 30, 14) ret (18, 30, 12)
PIRJ low R (18, 168) (18, 168)
enc (18, 30, 14) ret (18, 30, 12)
PIRJ low R (18, 168) (18, 168)
enc (18, 30, 9) ret (18, 30, 8)
PIRJ mid R (18, 72) (18, 72)
enc (18, 30, 9) ret (18, 30, 8)
PIRJ mid R (18, 72) (18, 72)
enc (18, 30, 9) ret (18, 30, 8)
PIRJ mid R (18, 72) (18, 72)
enc (18, 30, 9) ret (18, 30, 8)
PIRJ mid R (18, 72) (18, 72)
enc (18, 30, 9) ret (18, 30, 8)
PIRJ mid R (18, 72) (18, 72)
enc (18, 30, 9) ret (18, 30, 8)
PIRJ mid R (18, 72) (18, 72)
enc (18, 30, 20) ret (18, 30, 8)
PIRJ high R (18, 160) (18, 160)
enc (18, 30, 20) ret (18, 30, 8)
PIRJ high R (18, 160) (18, 160)
enc (18, 30, 20) ret (18, 30, 8)
PIRJ high R (18, 160) (18, 160)
enc (18, 30, 20) ret (18, 30, 8

### Create the TPS files for each condition

In [ ]:
from numpy.random import permutation
from itertools import product
from brainpipe.statistics import *

###############################################################################
st = study('Olfacto')
path_channels = join(st.path,'feature/TPSim_Enc_Ret_By_Odor/')
path_files = join(path_channels,'TPS_R_p_by_odor/')
path2save = join(path_channels, 'TPS_R_p_by_cond/')
###############################################################################
if not exists(path2save):
    makedirs(path2save)
###############################################################################

su_codes = ['S0','S1','S2','S3','S4','S5']
freqs = ['2_theta', '3_alpha', '4_beta','5_gamma1','6_gamma2']
method = ['s_Mail_RL']
su_odor_groups = {'CHAF' : {'low':['1','2','4','5'],
                              'high':['3','8','7','9']},
                    'VACJ' : {'low':['11','14','12','10'],
                              'high':['15','17','16','13']},
                    'SEMC' : {'low':['7','10','11','12','13'],
                              'high':['5','8','9']},
                    'PIRJ' : {'low':['1','9','5'],
                              'high':['4','6','7','18']},
                    'LEFC' : {'low':['15','2','1','16'],
                              'high':['14','3','4','17']},
                    'FERJ' : {'low':['7','2','16','17'],
                              'high':['12','1','5','13']},}

for su in (sorted(su_odor_groups)):
    for cond in su_odor_groups[su]:
        for freq in freqs:
            concat = np.array([])
            for odor in su_odor_groups[su][cond]:
                tps = np.load(path_files+'TPS_spear_'+su+'_odor_'+str(odor)+'_'+freq+'.npz')['tps']
                #print(su, freq, cond, odor, tps.shape)
                concat = np.concatenate((concat,tps), axis=1) if np.size(concat) else tps
            print(su, freq, cond, concat.shape)
            data = np.load(path_channels+su+'_odor'+str(odor)+'_E_pow'+freq+'.npz')
            mat = {'TPS':concat, 'label':data['label'],'channel':data['channel'],'xyz':data['xyz']}
            np.savez(path2save+'TPS_spear_'+su+'_odor_'+cond+'_'+freq+'.npz',**mat)

### Compute single-trial plots power to illustrate TPSim

In [ ]:
from os.path import join, exists
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, MaxNLocator
from brainpipe.visual import *

###############################################################################
st = study('Olfacto')
path_npz = join(st.path,'feature/TPSim_Enc_Ret_By_Odor_all/')
path2save = join(path_npz, 'plots/')
npz_enc = join(path_npz, '{}_odor{}_E_pow{}.npz')
npz_ret = join(path_npz, '{}_odor{}_R_pow{}.npz')
###############################################################################
if not exists(path2save):
    makedirs(path2save)
###############################################################################
th = '0.01'
method = ['s_Mail_RL']
freq = '0_theta'
odors_su = {'LEFC': {1:4,2:0,3:6,4:12,14:96,15:2,16:4,17:68},
            'PIRJ': {4:36,9:2,1:4,18:32,6:34,5:4,7:68}, #missing odor 15
            'FERJ': {16:6,17:6,5:8,7:6,12:8,13:8,2:6,1:10}}

for s,su in enumerate(sorted(odors_su)):
    for odor in sorted(odors_su[su]):
        mat_enc = np.load(npz_enc.format(su,odor,freq))
        mat_ret = np.load(npz_ret.format(su,odor,freq))
        pow_enc = mat_enc['xpow'][:,27:47,:]
        pow_ret = mat_ret['xpow'][:,27:47,:]
        label_E = mat_enc['label']
        label_R= mat_ret['label']
        time = np.arange(0,2,0.1)
        print('enc',pow_enc.shape,'ret',pow_ret.shape)
#         print('elecs names',label_E, label_R)
        if su == 'FERJ':
            elecs = [2] 
        elif su == 'LEFC':
            elecs = [8] 
        else:
            elecs = [3,10,12]
        for elec in elecs:
            for t0, t1 in product(range(pow_enc.shape[-1]),range(pow_ret.shape[-1])):
                pow0 = pow_enc[elec,:,t0]
                pow1 = pow_ret[elec,:,t1]
                max0, max1 = max(pow0), max(pow1)
                pow0, pow1 = pow0/max0, pow1/max1
                xfmt = ScalarFormatter(useMathText=True)
                rmaxis(plt.gca(), ['right', 'top'])
                xfmt.set_powerlimits((0,3))               
                rmaxis(plt.gca(), ['right', 'top'])
                plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))
                plt.plot(time, pow0, '-', color='purple', linewidth=2, label='Encoding')
                plt.plot(time, pow1, '-', color='orange', linewidth=2, label='Retrieval')
                plt.savefig(path2save+'plot_E_R_odor{}_{}_{}_trials{}_{}.png'.format(odor,su,elec,t0,t1))
                plt.savefig(path2save+'plot_E_R_odor{}_{}_{}_trials{}_{}.pdf'.format(odor,su,elec,t0,t1))
                plt.clf()
                plt.close()
            max0_2, max1_2 = np.max(pow_enc[elec]), np.max(pow_ret[elec])
            x = np.concatenate((pow_enc[elec]/max0_2,pow_ret[elec]/max1_2),axis=1)
            y = [0]*pow_enc.shape[-1] + [1]*pow_ret.shape[-1]
            BorderPlot(time, x, y=y, kind='sem', alpha=0.2, color=['purple','orange'],linewidth=2, 
                           ncol=1, xlabel='Time (s)',ylabel = r'Power', legend=['Encoding','Retrieval'])
            rmaxis(plt.gca(), ['right', 'top'])
            plt.legend(loc=0, handletextpad=0.1, frameon=False)
            plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))
            plt.savefig(path2save+'plot_E_R_odor{}_{}_{}_{}.pdf'.format(odor,su,elec,'mean'))            
            plt.savefig(path2save+'plot_E_R_odor{}_{}_{}_{}.png'.format(odor,su,elec,'mean'))
            plt.clf()
            plt.close()